## Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import os # Operational System
drive_dir = 'drive/MyDrive/'

# 'drive/MyDrive/datasets/glomerulos-classification-challenge/glomerulos/'
dataset_dir = os.path.join(drive_dir, 'datasets/glomerulos-classification-challenge/glomerulos/')
os.listdir(dataset_dir)

# Imports

In [ ]:
import os # Operational System
import time
import random

import numpy as np # matrices
import matplotlib.pyplot as plt # graphics and images

# PyTorch (Facebook)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# "PyTorch" for Images
import torchvision
from torchvision import datasets, transforms

## Transforms

In [ ]:
img_size = 224

# ImageNet mean and stdv
mean = [0.485, 0.456, 0.406] # R mean, G mean, B mean
stdv = [0.229, 0.224, 0.225] # R stdv, G stdv, B stdv

simple_train_transform = transforms.Compose([                                 
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean, stdv)
])

simple_val_transform = transforms.Compose([                                 
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean, stdv)
])

## Data sets

In [ ]:
root_dir = dataset_dir

my_datasets = {
    'train': datasets.ImageFolder(os.path.join(root_dir, 'train'), transform=simple_train_transform),
    'val': datasets.ImageFolder(os.path.join(root_dir, 'val'), transform=simple_val_transform)
}

In [ ]:
# select first image
image, label = my_datasets['train'][0]
lesion_dict = {0: 'endocapillary', 1: 'endomes', 2: 'mesangial', 3: 'normal'}
lesion = lesion_dict[label]

print(f'First image is labeled as {lesion}.')
np_img = [\
    image
    .detach()           # to remove the tensor from the computational graph
    .cpu()              # send image to cpu if it is in gpu
    .permute((1, 2, 0)) # [channels, height, width] -> [height, width, channels]
    .numpy()            # convert to numpy array
][0]

plt.imshow(np_img)

In [ ]:
# select a random image
random_idx = random.randint(0, len(my_datasets['val']))
image, label = my_datasets['val'][random_idx]
lesion = lesion_dict[label]

print(f'Image is of a {lesion}.')
np_img = [\
    image
    .detach()           # to remove the tensor from the computational graph
    .cpu()              # send image to cpu if it is in gpu
    .permute((1, 2, 0)) # [channels, height, width] -> [height, width, channels]
    .numpy()            # convert to numpy array
][0]

plt.imshow(np_img)

## Data loader

In [ ]:
batch_size = 64
num_workers = 2

my_dataloaders = {
    'train': DataLoader(
        my_datasets['train'],
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    ),
    'val': DataLoader(
        my_datasets['val'],
        batch_size=8*batch_size, # we are using a large batch size because we are not going to compute the gradients
        shuffle=False,
        num_workers=2,
        pin_memory=True
    ),
}

In [ ]:
# first option (usually the worst)

# To understand: images, labels = iter(my_dataloaders['train']).next()
dataiter = iter(my_dataloaders['train'])
for _ in range(len(my_dataloaders['train']) // batch_size):
    images, labels = dataiter.next() # we are loading now a BATCH of images and labels
    print(images.shape) # [batch, channel, height, width]

    # select only the first image and plot it
    np_img = images[0].detach().cpu().permute((1, 2, 0)).numpy()
    plt.imshow(np_img)
    break

In [ ]:
# second option (usually the best)
for images, labels in my_dataloaders['train']:
    print(images.shape) # [batch, channel, height, width]

    # select only the first image and plot it
    np_img = images[0].detach().cpu().permute((1, 2, 0)).numpy()
    plt.imshow(np_img)
    break

## Model

In [ ]:
model = torchvision.models.resnet50(pretrained=True)
model.fc = nn.Linear(in_features=2048, out_features=4, bias=True)

## Custom Model

In [ ]:
class RedeVisaoComputacional(nn.Module):
    def __init__(self):
        super().__init__()
        # Define your network architecture below
        self.fc1 = nn.Linear(in_features=3*224*224, out_features=10)
        self.fc2 = nn.Linear(in_features=10, out_features=4)

    def forward(x):
        # Compute forward pass below
        x = self.fc1(x)
        x = self.fc2(x)

        return x

## Other Inputs

In [ ]:
epochs = 300 # steps
learning_rate = 1e-4

criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Training

In [ ]:
def single_gpu_train(
    model,
    train_loader,
    valid_loader,
    criterion,
    optimizer,
    name='visao-computacional',
    num_classes=4,
    device='cuda'
):
    device = torch.device(device)
    model = model.to(device)

    min_valid_loss = np.Inf
    for epoch in range(1, epochs+1):
        print(f"Epoch {epoch}/{epochs}\n")
        start_time = time.time()

        ############
        # Training #
        ############
        train_running_loss = 0
        train_class_correct = [0. for _ in range(num_classes)] # [0., 0., 0., 0.] if four classes
        train_class_total = [0. for _ in range(num_classes)]   # [0., 0., 0., 0.] if four classes
        for images, labels in train_loader:
            # put model in training mode
            model.train()

            # send images and labels to GPU if available
            images, labels = images.to(device), labels.to(device)

            # compute the feedforward pass and the loss
            log_ps = model(images) # feedforward
            loss = criterion(log_ps, labels)
            train_running_loss += loss.item()

            # update the weights with back propagation
            optimizer.zero_grad() # zero the gradients
            loss.backward()       # backprop
            optimizer.step()      # update the weights

            # convert output probabilities to the predicted class
            _, pred = torch.max(log_ps, 1)

            # compare predictions to true label
            correct_tensor = pred.eq(labels.data.view_as(pred))
            correct = np.squeeze(correct_tensor.cpu().numpy())

            # calculate train accuracy for each object class
            for i in range(images.shape[0]):
                label = labels.data[i]
                train_class_correct[label] += correct[i].item()
                train_class_total[label] += 1

        # Training metrics
        avg_training_loss = train_running_loss/len(train_loader.dataset)
        print(f"Training loss: {avg_training_loss:.3f}")

        true_positives = int(np.sum(train_class_correct))
        total = int(np.sum(train_class_total))
        percentage = 100*true_positives/total
        print(f'Train Accuracy (Overall): {percentage:.2f}% ({true_positives}/{total})\n')

        ########################################
        # Evaluation on the validation dataset #
        ########################################
        with torch.no_grad():
            accuracy, running_valid_loss = 0, 0
            class_correct = [0. for _ in range(num_classes)]
            class_total = [0. for _ in range(num_classes)]
            for images, labels in valid_loader:
                # put model in evaluation mode
                model.eval() # For instance, remove dropout

                # send images and labels to GPU if available
                images, labels = images.to(device), labels.to(device)

                # compute the loss for tracking purposes (we do not backprogate)
                log_ps = model(images)
                batch_loss = criterion(log_ps, labels)
                running_valid_loss += batch_loss.item()

                # Calculate accuracy
                ps = torch.exp(log_ps)
                _, top_class = ps.topk(1, dim=1)
                equals = (top_class == labels.view(*top_class.shape))
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

                # convert output probabilities to predicted class
                _, pred = torch.max(log_ps, 1)    
                # compare predictions to true label
                correct_tensor = pred.eq(labels.data.view_as(pred))
                correct = np.squeeze(correct_tensor.cpu().numpy())

                # calculate test accuracy for each object class
                for i in range(images.shape[0]):
                    label = labels.data[i]
                    class_correct[label] += correct[i].item()
                    class_total[label] += 1

        # Validation metrics
        avg_validation_loss = running_valid_loss/len(valid_loader.dataset)
        print(f"Validation loss: {avg_validation_loss:.3f}")

        true_positives = int(np.sum(class_correct))
        total = int(np.sum(class_total))
        percentage = 100*true_positives/total
        print(f'Validation Accuracy (Overall): {percentage:.2f}% ({true_positives}/{total})')

        # save model if validation loss decreased
        if avg_validation_loss < min_valid_loss:
            print(f'Validation loss decreased ({min_valid_loss:.6f} --> {avg_validation_loss:.6f}). Saving model ...')
            torch.save(model.state_dict(), f'{name}-checkpoint.pth')
            min_valid_loss = avg_validation_loss

        end_time = time.time()

        print(f"Epoch took {(end_time - start_time):.3f} seconds\n")

## Training per se

In [ ]:
single_gpu_train(
    model=model,
    train_loader=my_dataloaders['train'],
    valid_loader=my_dataloaders['val'],
    criterion=criterion,
    optimizer=optimizer
)

## What's next?

- Activation Functions
- Advanced Data Augmentation
- Batch Normalization
- Dropout
- Input Size
- Learning Rate Schedulers, Linear Warm-Up
- Loss Function (Weights)
- Metric(s)
- Optimizers
- Avg. Pooling; 
- Etc.